In [1]:
import os 
import glob
import pandas as pd 
import numpy as np
from utils.linkageTree import linkageCut
from tsp.TSP_Formulation_Methods import ( 
    create_QUBO_matrix,
    solve_qubo_with_Dwave,
    check_solution, 
    load_lambda_means,
    draw_solution_graph,
)
from vqaa.vqaa_tools import ( 
    heuristical_embedding, 
    atoms_register,
    atoms_list, 
    generate_grid,
    run_vqaa,
    plot_distribution,
)
from utils.utils import ( 
    view_linkage_on_map, 
    draw_centers_on_map,
    map_draw_line,
    convert_bitstring_to_matrix
)

/home/yllari/All/Environments/Pasqal_new/lib/python3.10/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/yllari/All/Environments/Pasqal_new/lib/python3.10/site-packages/torch/cuda/__init__.py:905: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


# Load initial data 

In [2]:
# Load previously stored overpy lat/lon datafile for different amenities
amenities_data = pd.read_csv(os.path.join(os.pardir, 'data', 'amenities-granada.csv'))
# If there is no previous data:
# amenities_data = utils.fetch_amenities_from() # Defaults to Granada
# Create a hierarchical clustering of amenities
hierarchical_cluster = linkageCut(amenities_data)
# Set a specific number of clusters per levels
nclusters = 4
levels = 2
labels = hierarchical_cluster.top_down_view_recur(nclusters=nclusters, levels=2)
# Visualize for debugging purposes.
view_linkage_on_map(linkage_matrix=hierarchical_cluster)

# First iteration

In [3]:
# Fetch the centers of the first level
centers = hierarchical_cluster.give_centers_level(0)
# Sanity check by drawing the graph
draw_centers_on_map(centers, color='red')

In [4]:
# Fetch the distance from the centers of the first level
distances = hierarchical_cluster.dist_matrix_level(0, return_labels=False)
# Set initial global parameters
N = distances.shape[0]
p = 2

startNode = 0
endNode = np.random.choice(np.arange(distances.shape[0] - 1) + 1)

# Process Parameters
p = min(p, N-1)
startNode = min(startNode, N-1)
endNode = min(endNode, N-1)

reduced_distances = distances[:N,:N]/np.max(distances[:N,:N])
maxDistance = np.max(reduced_distances)
# NOTE: temporary double pardir while we decide the new structure for the lambdas
lambda_paths = glob.glob(os.path.join(os.path.pardir, os.path.pardir, 'data', 'lamdasOptimized', '*'))
mean_lambdas = load_lambda_means(lambda_paths)

# Formulation with initial lambdas
Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode, endNode, mean_lambdas)

In [5]:
solution_Dwave, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
print("\nD-Wave solution:")
print(solution_Dwave)
check_solution(solution_Dwave, N,  p, startNode, endNode)


D-Wave solution:
[1 0 0 0 0 1 0 0 0 0 1 0]


In [6]:
adjacency = convert_bitstring_to_matrix(solution_Dwave, N=N, p=p)
map_draw_line(centers[:, ::-1], adjacency, color='red')

In [7]:
# coords = heuristical_embedding(atoms_list(len(Q_matrix_initial)), generate_grid(50, 50,1), Q_matrix_initial)
# register = atoms_register(coords)

In [8]:
# Why is it takiung > 5minutes????
# C, x = run_vqaa(Q_matrix_initial, register, "mps")

# Second iteration

In [11]:
# Formulation with initial lambdas
sol_bitstrings = []

all_indices = set(np.arange(nclusters - 1) + 1)
print(adjacency)
for i in range(1, nclusters+1):
    print("solving", i)
    connections = np.concatenate([adjacency[:,i-1].nonzero()[0], adjacency[i-1, :].nonzero()[0]], axis=0) + 1
    
    print("connections", connections)
    if len(connections) > 0: #Selected
        # Fetch the centers of the first level
        distances, indices, _ = hierarchical_cluster.dist_matrix_label_down(
        i,
        connections=connections,
        )
        print(indices)
        startNode = None
        if len(indices) >= 1:
            startNode = indices[0]
            choices = all_indices - set([startNode])
            print(choices)
        if len(indices) == 2:
            endNode = indices[1]
        else:
            endNode = np.random.choice(list(choices))
        print("Start-end", startNode, endNode)

        Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode, endNode, mean_lambdas)
        solution_Dwave, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
        print("\nD-Wave solution:")
        print(solution_Dwave)
        check_solution(solution_Dwave, N,  p, startNode, endNode)
        sol_bitstrings.append(solution_Dwave)
    else:
        sol_bitstrings.append(np.zeros((nclusters*(p+1))))
next_adjacency = convert_bitstring_to_matrix(solution_Dwave, N=N, p=p)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
solving 1
connections [2]
[2]
{1, 3}
Start-end 2 1

D-Wave solution:
[0 0 1 0 1 0 0 0 0 1 0 0]
solving 2
connections [1 3]
[2, 2]
{1, 3}
Start-end 2 2

D-Wave solution:
[0 0 1 0 1 0 0 0 0 0 1 0]
Constraint 3 not fulfilled. Some stops are visited more than once.
solving 3
connections [2]
[2]
{1, 3}
Start-end 2 1

D-Wave solution:
[0 0 1 0 1 0 0 0 0 1 0 0]
solving 4
connections []


In [12]:
from utils.utils import assemble_line
assembled = assemble_line(sol_bitstrings,[], nclusters, p)

In [13]:
next_centers = hierarchical_cluster.give_centers_label_down(1)
map = map_draw_line(centers[:, ::-1], adjacency, color='red')
map_draw_line(next_centers[:, ::-1], next_adjacency, color='blue', map=map)

In [16]:
centers_level1 = hierarchical_cluster.give_centers_level(1)
map_draw_line(centers_level1[:,::-1], assembled, color='blue')